In [2]:
from wandb_import import *

In [3]:
from pathlib import Path
import sys

yolo_dir = Path.home()/'new_yolov5/yolov5'
if yolo_dir.is_dir():
    sys.path.append(str(yolo_dir))

from utils.tools.general import *

In [4]:
!wandb login

wandb: Currently logged in as: silkron_cy (use `wandb login --relogin` to force relogin)


In [4]:
'''mount disk for `lightbox_china_train` data'''



def format_disk(device_name):
	!sudo mkfs.ext4 -m 0 -E lazy_itable_init=0,lazy_journal_init=0,discard /dev/{device_name}

def mount_disk(device_name, mount_dir):
	!sudo mkdir -p /mnt/disks/{mount_dir}
	!sudo mount -o discard,defaults /dev/{device_name} /mnt/disks/{mount_dir}
	!sudo chmod a+w /mnt/disks/{mount_dir}

mount_disk('sdb', 'lightbox_china_train')
mount_disk('sdc', 'lightbox_china_merged-val')
mount_disk('sdd', 'lightbox_hand_eccsd-copy')
mount_disk('sde', 'coco-2017-2')
mount_disk('sdf', 'pasted-val')
!sudo lsblk 

mount: /mnt/disks/lightbox_china_train: /dev/sdb already mounted on /mnt/disks/lightbox_china_train.
mount: /mnt/disks/lightbox_china_merged-val: /dev/sdc already mounted on /mnt/disks/lightbox_china_merged-val.
mount: /mnt/disks/lightbox_hand_eccsd-copy: /dev/sdd already mounted on /mnt/disks/lightbox_hand_eccsd-copy.
mount: /mnt/disks/coco-2017-2: /dev/sde already mounted on /mnt/disks/coco-2017-2.
mount: /mnt/disks/pasted-val: /dev/sdf already mounted on /mnt/disks/pasted-val.
NAME    MAJ:MIN RM   SIZE RO TYPE MOUNTPOINT
loop0     7:0    0  55.5M  1 loop /snap/core18/2246
loop1     7:1    0  55.5M  1 loop /snap/core18/2253
loop2     7:2    0  61.9M  1 loop /snap/core20/1169
loop3     7:3    0  61.9M  1 loop /snap/core20/1242
loop4     7:4    0 251.7M  1 loop /snap/google-cloud-sdk/206
loop5     7:5    0 251.7M  1 loop /snap/google-cloud-sdk/207
loop6     7:6    0  67.2M  1 loop /snap/lxd/21803
loop7     7:7    0  67.2M  1 loop /snap/lxd/21835
loop8     7:8    0  42.2M  1 loop /snap/

# create filelist txt for val

In [5]:
# yaml_dir = yolo_dir/'data'/'test_merge_training'
# yaml_dir = yolo_dir/'data'/'fake_coco128'
yaml_dir = yolo_dir/'data'/'sampled_coco2017'
yaml_dir.mkdir(parents=True, exist_ok=True)



## train

In [6]:
# train_dpath = (yolo_dir/'datasets'/'coco128_no_label')
# train_dpath = yolo_dir/'utils/paster/examples/fake_coco_128'
train_dpath = Path('/mnt/disks/coco-2017-2/coco2017/train2017')
train_dpath.lls()

102270

In [122]:


# train_img_df = create_path_df((train_dpath/'images'/'train2017').ls())
# train_lbl_df = create_path_df((train_dpath/'labels'/'train2017').ls())
train_img_df = create_path_df((train_dpath/'images').ls())
train_lbl_df = create_path_df((train_dpath/'labels').ls())

train_img_df = train_img_df[train_img_df['is_file']]
train_lbl_df = train_lbl_df[train_lbl_df['is_file']]
train_img_and_lbl_df, columns_to_show = join_img_and_lbl_df(train_img_df, train_lbl_df)

# train_img_and_lbl_df.head()
sampled_train_df = train_img_and_lbl_df.sample(120)
train_img_paths = sampled_train_df['file_path-img'].values
fu.f_writelines(train_img_paths, yaml_dir/'train_sampled_images.txt')

  0%|          | 0/19222 [00:00<?, ?it/s]

  0%|          | 0/19222 [00:00<?, ?it/s]

In [66]:
# all_imgs = pd.Series((train_dpath/'images').ls())
# found_img = train_img_and_lbl_df['file_path-img']
# all_imgs[~all_imgs.isin(found_img)]

# others = (train_dpath.parent/'train2017_others')
# all_imgs[~all_imgs.isin(found_img)].apply(lambda x: Path(x).rename( others/Path(x).name))
# train_img_and_lbl_df['parent-img'][~train_img_and_lbl_df['parent-img'].isin((train_dpath/'images').ls())]

1         /mnt/disks/coco-2017-2/coco2017/train2017/imag...
2         /mnt/disks/coco-2017-2/coco2017/train2017/imag...
3         /mnt/disks/coco-2017-2/coco2017/train2017/imag...
4         /mnt/disks/coco-2017-2/coco2017/train2017/imag...
6         /mnt/disks/coco-2017-2/coco2017/train2017/imag...
                                ...                        
101262    /mnt/disks/coco-2017-2/coco2017/train2017/imag...
101263    /mnt/disks/coco-2017-2/coco2017/train2017/imag...
101265    /mnt/disks/coco-2017-2/coco2017/train2017/imag...
101266    /mnt/disks/coco-2017-2/coco2017/train2017/imag...
101267    /mnt/disks/coco-2017-2/coco2017/train2017/imag...
Length: 82047, dtype: object

## val

In [121]:
# val_dpath = Path('/mnt/disks/lightbox_china_merged-val/data')
# val_dpath = train_dpath
val_dpath = Path('/mnt/disks/coco-2017-2/coco2017/test2017')

val_img_df = create_path_df((val_dpath/'images').ls())
val_lbl_df = create_path_df((val_dpath/'labels').ls())

val_img_df = val_img_df[val_img_df['is_file']]
val_lbl_df = val_lbl_df[val_img_df['is_file']]
val_img_and_lbl_df, columns_to_show = join_img_and_lbl_df(val_img_df, val_lbl_df)

sampled_val_df = val_img_and_lbl_df.sample(40)
val_img_paths = sampled_val_df['file_path-img'].values
fu.f_writelines(val_img_paths, yaml_dir/'val_sampled_images.txt')

  0%|          | 0/40670 [00:00<?, ?it/s]

  0%|          | 0/5298 [00:00<?, ?it/s]

/tmp/ipykernel_35650/2523803976.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_lbl_df = val_lbl_df[val_img_df['is_file']]


# create yaml

In [120]:
train_txt = yaml_dir/'train_sampled_images.txt'
val_txt = yaml_dir/'val_sampled_images.txt'
data_yaml = yaml_dir/'lightbox_test_data.yaml'

str(train_txt), str(val_txt)

('/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/train_sampled_images.txt',
 '/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/val_sampled_images.txt')

In [106]:
'''remove labels cache'''

for lbl_cache in yaml_dir.ls('*.cache'):
    print(lbl_cache)
    lbl_cache.unlink()

In [107]:
# %%writefile {data_yaml}

# # Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
# path: /home/cheeyung/yolov5/datasets/coco128  # dataset root dir
# train: images/train2017  # train images (relative to 'path') 128 images
# val: /home/cheeyung/yolov5/data/test_training/val_sampled_images.txt  # val images (relative to 'path') 128 images

# # Classes
# nc: 80  # number of classes
# names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
#         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
#         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
#         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
#         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
#         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
#         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
#         'hair drier', 'toothbrush']  # class names


# # Download script/URL (optional)


In [108]:
# %%writefile {data_yaml}


# # path: /home/cheeyung/yolov5/datasets/coco128  # dataset root dir
# train: /home/cheeyung/yolov5/data/test_training/train_sampled_images.txt
# val: /home/cheeyung/yolov5/data/test_training/val_sampled_images.txt

# # # Classes
# nc: 80  # number of classes
# names: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
#         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
#         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
#         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
#         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
#         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
#         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
#         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
#         'hair drier', 'toothbrush']  # class names


# # # Download script/URL (optional)


In [110]:
yaml_dir

PosixPath('/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017')

In [123]:
train_txt = yaml_dir/'train_sampled_images.txt'
val_txt = yaml_dir/'val_sampled_images.txt'

str(train_txt), str(val_txt), train_txt.is_file(), val_txt.is_file()


('/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/train_sampled_images.txt',
 '/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/val_sampled_images.txt',
 True,
 True)

In [130]:
%%writefile {data_yaml}

train: /home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/train_sampled_images.txt
val:  /home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/val_sampled_images.txt


nc: 14

names:  ['wangzai_milk',
 'wangwang_coco_vanilla',
 'wangwang_coco_orange',
 'vitamilk',
 'lixing_yogurt_strawberry',
 'kangshifu_green_tea',
 'tongyi_milktea',
 'kangshifu_beefNoodle',
 'wanglaoji',
 'sprite_200',
 'pocky_coco',
 'lixing_yogurt_kiwi',
 'jiaduobao_550ml',
 'kangshifu_peach_drink']

Overwriting /home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/lightbox_test_data.yaml


# hyp yaml

In [125]:
hyp_custom_yaml = yaml_dir/'hyp_custom.yaml'

In [134]:
%%writefile {hyp_custom_yaml}
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.05  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.05  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.3  # image flip left-right (probability)
mosaic: 0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)


Overwriting /home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/hyp_custom.yaml


In [126]:
# %%writefile {hyp_custom_yaml}
# # Hyperparameter Evolution Results
# # Generations: 50
# # Metrics:      0.519     0.968     0.938     0.657    0.0108   0.00224   0.00387

# lr0: 0.00428
# lrf: 0.106
# momentum: 0.878
# weight_decay: 0.00036
# warmup_epochs: 1.08
# warmup_momentum: 0.547
# warmup_bias_lr: 0.0471
# box: 0.0264
# cls: 0.263
# cls_pw: 1.01
# obj: 0.301
# obj_pw: 1.25
# iou_t: 0.3
# anchor_t: 3.29
# # anchors: 3.53
# fl_gamma: 0.0
# hsv_h: 0.01
# hsv_s: 0.2
# hsv_v: 0.2
# degrees: 0.876
# translate: 0.05
# scale: 0.05
# shear: 0.05
# perspective: 0.0
# flipud: 0.3
# fliplr: 0.3
# mosaic: 0
# mixup: 0.0
# copy_paste: 0.0  # segment copy-paste (probability)

Overwriting /home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/hyp_custom.yaml


# training

In [115]:
def ci_test_train(img_size):
    '''continuous intergration testing '''

    !python train.py --project {PROJECT} \
        --name {save_name} \
        --hyp {hyp_custom_yaml} \
        --data {data_yaml} \
        --cfg {cfg_yaml} \
        --epochs {epochs} \
        --batch-size {batch_size} \
        --img-size {img_size} \
        # --save_period & \
        --noautoanchor

def training_cli():
    !python train.py --project {PROJECT} \
        --name {save_name} \
        --hyp {hyp_custom_yaml} \
        --data {data_yaml} \
        --cfg {cfg_yaml} \
        --epochs {epochs} \
        --batch-size {batch_size} \
        --img-size {img_size} \
        --save_period 2 \
        --cache

In [127]:
hyp_custom_yaml = hyp_custom_yaml.resolve()
cfg_yaml = (yolo_dir/'models/yolov5m.yaml').resolve()
data_yaml = data_yaml.resolve()

%cd {yolo_dir}

(hyp_custom_yaml, cfg_yaml, data_yaml)

/home/cheeyung/new_yolov5/yolov5


(PosixPath('/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/hyp_custom.yaml'),
 PosixPath('/home/cheeyung/new_yolov5/yolov5/models/yolov5m.yaml'),
 PosixPath('/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/lightbox_test_data.yaml'))

In [128]:
# !python train.py --help

In [135]:
!python train.py --img 320 --batch 16 --weights yolov5n.pt --cfg yolov5n.yaml --epochs 1 --device 'cpu' --data {data_yaml} --hyp {hyp_custom_yaml}

wandb: Currently logged in as: silkron_cy (use `wandb login --relogin` to force relogin)
train: weights=yolov5n.pt, cfg=yolov5n.yaml, data=/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/lightbox_test_data.yaml, hyp=/home/cheeyung/new_yolov5/yolov5/data/sampled_coco2017/hyp_custom.yaml, epochs=1, batch_size=16, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=cpu, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 114 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-2-g3a3aca8 torch 1.7.0+cu110 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, 

In [24]:
!python train.py --help | grep hyp

wandb: Currently logged in as: silkron_cy (use `wandb login --relogin` to force relogin)
usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
  --hyp HYP             hyperparameters path
  --evolve [EVOLVE]     evolve hyperparameters for x generations


In [23]:
'''jump'''
# img_size = 1024

img_size = 640
# batch_size = 32 if img_size < 1024 else 16
batch_size = 4
print(f"batch_size:  {batch_size}")


PROJECT = 'fake_coco128--debug'
# PROJECT = 'lightbox_val_merged--debug'
save_name = 'v6_master--hsv'
# save_name = f'labeled_coco128_sample120--{img_size}-'
epochs = 1

ci_test_train(img_size)
        

batch_size:  4
wandb: Currently logged in as: silkron_cy (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=/home/cheeyung/new_yolov5/yolov5/models/yolov5m.yaml, data=/home/cheeyung/new_yolov5/yolov5/data/fake_coco128/lightbox_test_data.yaml, hyp=/home/cheeyung/new_yolov5/yolov5/data/fake_coco128/hyp_custom.yaml, epochs=1, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=fake_coco128--debug, name=v6_master--hsv, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 114 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.0-2-g3a3aca8 torch 1.7.0+

In [ ]:
def plot_train_img(project, save_name):
    result_dirs = (yolo_dir/project).ls(save_name+'*')
    
    if not any(p.is_dir() for p in result_dirs):
        print('Result Directory not found')
    else:
        latest_run = sorted(result_dirs)[-1]
        img_paths = latest_run.ls('train_batch*.jpg')

        if len(img_paths):
            print('train img found')
            [ia.imshow(imageio.imread(im)) for im in img_paths]
    

plot_train_img(PROJECT, save_name)


# debug

## parse arg for debug training in notebook

In [ ]:
# from test_train import *


# parse_arg_notetbook = (yolo_dir/'paste_product'/'parse_args.ipynb')
# if parse_arg_notetbook.is_file():
#     !jupyter nbconvert --to script {parse_arg_notetbook} 
# else:
#     raise ValueError('Notebook not found')

# from paste_product.parse_args import get_default_args, merge_args, parse_args, print_if_keys_in_dict

In [ ]:
def init_args(train_options, file_path_args):
    merged_train_args = merge_args(train_options, file_path_args)

    print_if_keys_in_dict(train_options, merged_train_args)
    print_if_keys_in_dict(file_path_args, merged_train_args)
    print_if_keys_in_dict(get_default_args(), merged_train_args)

    train_namespace = parse_args(train_options, file_path_args)

    return train_namespace

def parse_debug_opt(train_options, file_path_args):
    opt = init_args(train_options, file_path_args)
    opt.resume = False
    return opt

def training_with_opt_in_nb():

    def debug_training():
        opt = parse_debug_opt(train_options, file_path_args)
        print(opt)
        main(opt)



    file_path_args = {
        'hyp' : hyp_custom_yaml,
        'data' : data_yaml,
        'weights' : yolo_dir/'yolov5s.pt',
        'cfg' : yolo_dir/ 'models/yolov5s.yaml'
    }


    train_options = {
        'project' : 'lightbox_val_merged--debug',
        # 'name' : 'solved_filter_front_img_in_dataset',
        'name' : 'img_not_log_in_notebook_arg',

        'epochs' : 1,
        'batch_size' : 16,
        'imgsz' : 160,
        'nosave' : False,
        'upload_dataset': False,
        'save_period': -1,
    }

    # init_args(train_options, file_path_args)
    debug_training()

# training_with_opt_in_nb()

## json dump namespace to file

In [ ]:

# import json

# def dump_nampespace_to_file(namespace_opt, save_file='opt_dict.txt'):
#     opt_dict = vars(namespace_opt)
#     for k,v in opt_dict.items():
#         if isinstance(v, Path):
#             opt_dict[k] = str(v)
#     with open(save_file, 'w') as f:
#         json.dump(opt_dict, f)


# dump_nampespace_to_file(parse_debug_opt(train_options, file_path_args))

NameError: name 'train_options' is not defined

In [ ]:
'''
dump_nampespace_to_file -> train_cli_opt
init_args(train_options, file_path_args) -> notebook_opt

compare_with_df(train_cli_opt, vars(notebook_opt))
'''

# test_db_opt = init_args(train_options, file_path_args)
# pprint(vars(test_db_opt))

# col = 'hyp'
# print(f"opt_df['ori'][{col}]:  {opt_df['ori'][col]}")
# f"opt_df['mine'][{col}]:  {opt_df['mine'][col]}"

# with open('opt_dict.txt', 'r') as f:
#     json_ori_train_opt = json.load(f)

# opt_df = pd.DataFrame([json_ori_train_opt, vars(test_db_opt)], index=['ori', 'mine']).T
# has_same_opt = (opt_df['ori'] == opt_df['mine'])
# opt_df[~has_same_opt]

'\ndump_nampespace_to_file -> train_cli_opt\ninit_args(train_options, file_path_args) -> notebook_opt\n\ncompare_with_df(train_cli_opt, vars(notebook_opt))\n'

## debug test
(nid to remove label's .cache file)

- real labeled coco128
    - by dir (all img) = pass
    - by txt (sample 10) = train not upload, val img pass
    - by txt (sample 120) 
        - im_size 320 = pass
        - im_size 160 = pass

- fake labeled coco128 (dummy label with only 13 class)
    - by txt (sample 120) 
        - im_size 160 = pass

